In [145]:
#TO RE-RUN
%reset -f

In [146]:
from sklearn import preprocessing
from time import time
import numpy as np
import csv
from sklearn import metrics
from sklearn.preprocessing import scale
from sklearn.feature_selection import VarianceThreshold
from sklearn.cross_validation import StratifiedShuffleSplit, cross_val_score

from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import BernoulliNB, MultinomialNB, GaussianNB

from sklearn.grid_search import GridSearchCV, ParameterGrid
from sklearn.preprocessing import StandardScaler

from imblearn.over_sampling import SMOTE,ADASYN, RandomOverSampler
from imblearn.pipeline import Pipeline
from imblearn.pipeline import make_pipeline

from operator import truediv
from sklearn import metrics
import pandas as pd
import time
import os

from pylab import *
import seaborn as sns
import matplotlib.pyplot as plt


np.set_printoptions(suppress=True)
pd.options.display.float_format = '{:,.2f}'.format
plt.style.use('classic')

%matplotlib inline

import sys
sys.path.insert(1, "/home/ilmira/healthforecast/readmission/src/")
from TypeFeatImputer import TypeFeatImputer
from UnivCombineFilter import UnivCombineFilter

In [147]:
typeEncounter = "first" #["last","first"]

In [148]:
#df_all=pd.read_csv(os.path.join('resources','diabetic_data_processed_withweight.csv'),';')
df_all=pd.read_pickle(os.path.join('resources','clean_data_' + typeEncounter + '.pkl'))
print df_all.shape
print df_all.columns.tolist()
print df_all.readmitted.value_counts()
print df_all.readmitted.value_counts()/float(df_all.shape[0])

(71518, 174)
['gender', 'age', 'weight', 'time_in_hospital', 'num_lab_procedures', 'num_procedures', 'num_medications', 'number_outpatient', 'number_emergency', 'number_inpatient', 'number_diagnoses', 'max_glu_serum', 'A1Cresult', 'metformin', 'repaglinide', 'nateglinide', 'chlorpropamide', 'glimepiride', 'acetohexamide', 'glipizide', 'glyburide', 'tolbutamide', 'pioglitazone', 'rosiglitazone', 'acarbose', 'miglitol', 'troglitazone', 'tolazamide', 'examide', 'citoglipton', 'insulin', 'glyburide-metformin', 'glipizide-metformin', 'glimepiride-pioglitazone', 'metformin-rosiglitazone', 'metformin-pioglitazone', 'Change', 'diabetesMed', 'Diabetis', 'Infectious and parasitic diseases', 'Neoplasms', 'Endocrine', 'Blood', 'Mental', 'Nervous', 'Organs', 'Circulatory', 'Respiratory', 'Digestive', 'Genitourinary', 'Pregnancy', 'Skin', 'Muscoskeletal', 'Congenital', 'Perinatal', 'Ill-defined', 'medSpec_AllergyandImmunology', 'medSpec_Anesthesiology', 'medSpec_Anesthesiology-Pediatric', 'medSpec_C

### Features preparation

In [149]:
df_all["HbA1c"] = -1
df_all["HbA1c"][df_all.A1Cresult == 2] = 3
df_all["HbA1c"][df_all.A1Cresult == 1] = 2
df_all["HbA1c"][df_all.A1Cresult == 0] = 1
df_all["HbA1c"][df_all.A1Cresult.isnull()] = 0

pd.value_counts(df_all.HbA1c)

/home/ilmira/.conda/envs/readmision/lib/python2.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/ilmira/.conda/envs/readmision/lib/python2.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/ilmira/.conda/envs/readmision/lib/python2.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  afte

0    56130
3     7271
1     4642
2     3475
Name: HbA1c, dtype: int64

In [150]:
df_all["age_cat"] = -1
df_all["age_cat"][df_all.age.isin([0,1,2])] = 0
df_all["age_cat"][df_all.age.isin([3,4,5,6])] = 1
df_all["age_cat"][df_all.age.isin([7,8,9,10])] = 2

print pd.value_counts(df_all["age_cat"])

1    37663
2    32066
0     1789
Name: age_cat, dtype: int64


/home/ilmira/.conda/envs/readmision/lib/python2.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/ilmira/.conda/envs/readmision/lib/python2.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/ilmira/.conda/envs/readmision/lib/python2.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  afte

In [151]:
df_all["adm_src_ref"] = np.sum(df_all[["adm_src_1","adm_src_2","adm_src_3"]].values,axis=1)>0
df_all["adm_src_em"] = df_all["adm_src_7"].values

print pd.value_counts(df_all["adm_src_ref"])
print pd.value_counts(df_all["adm_src_em"])

False    49132
True     22386
Name: adm_src_ref, dtype: int64
1    38844
0    32674
Name: adm_src_em, dtype: int64


In [152]:
df_all["medSpec_cardio"] = np.sum(
    df_all[['medSpec_Cardiology', 'medSpec_Cardiology-Pediatric']], axis=1)>0

pd.value_counts(df_all["medSpec_cardio"])

False    67140
True      4378
Name: medSpec_cardio, dtype: int64

In [153]:
df_all["medSpec_surgery"] = np.sum(
    df_all[['medSpec_Surgeon', 'medSpec_Surgery-Cardiovascular', 'medSpec_Surgery-Cardiovascular/Thoracic', 
            'medSpec_Surgery-Colon&Rectal', 'medSpec_Surgery-General', 'medSpec_Surgery-Maxillofacial', 
            'medSpec_Surgery-Neuro', 'medSpec_Surgery-Pediatric', 'medSpec_Surgery-Plastic', 
            'medSpec_Surgery-Thoracic', 'medSpec_Surgery-Vascular', 
            'medSpec_SurgicalSpecialty', 'medSpec_Urology']], axis=1)>0

pd.value_counts(df_all["medSpec_surgery"])

False    66968
True      4550
Name: medSpec_surgery, dtype: int64

In [154]:
cols = ["diss_1",
        "race_AfricanAmerican","race_Caucasian","race_Other",
        "medSpec_cardio","medSpec_Family/GeneralPractice","medSpec_InternalMedicine","medSpec_surgery",
        "age_cat",
        "Diabetis","Circulatory",'Digestive','Genitourinary','Poisoning','Muscoskeletal','Neoplasms','Respiratory',
        'HbA1c',
        "Change",
        'time_in_hospital'        
       ]

In [155]:
colsExtended = cols[:]
colsExtended.extend(['num_lab_procedures', 'num_procedures', 'num_medications', 'number_outpatient', 
            'number_emergency', 'number_inpatient', 'number_diagnoses',"readmitted"])
cols.extend(["readmitted"])


print len(cols), len(colsExtended)

21 28


#### Save

In [157]:
print typeEncounter, df_all[cols].shape
df_all[cols].to_pickle(os.path.join('resources','clean_data_' + typeEncounter + '_hyp_1.pkl'))

first (71518, 21)


In [156]:
print typeEncounter,df_all[colsExtended].shape
df_all[colsExtended].to_pickle(os.path.join('resources','clean_data_' + typeEncounter + '_hyp_1_extended.pkl'))

first (71518, 28)
